# Introduction
This notebook is for building and testing a pipeline for creating synthetic data with GANs

### Imports
Import libraries and write settings here.

In [13]:
from plotly.offline import iplot
import plotly.offline as py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
from IPython import get_ipython
import cufflinks as cf
import os
import sys
import zipfile

import importlib

# models
import sklearn.cluster as cluster

from ydata_synthetic.synthesizers.regular import WGAN_GP, VanilllaGAN
from ydata_synthetic.preprocessing.regular.credit_fraud import *

#from config import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
InteractiveShell.ast_node_interactivity = 'all'

ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
py.init_notebook_mode(connected=True)

cf.go_offline(connected=True)
cf.set_config_file(theme='white')

#### Set data paths

In [25]:
external_data = '../data/external/'
raw_data = '../data/raw/'

#### Import data from Kaggle

In [45]:
# downloading from https://www.kaggle.com/mlg-ulb/creditcardfraud
# we write to the raw data directory path with './'
api.dataset_download_files('dansbecker/hospital-readmissions',
                          path=external_data)

In [47]:
#unzip

with zipfile.ZipFile(external_data+'hospital-readmissions.zip', 'r') as zipref:
    zipref.extractall(raw_data)

#### Create kaggle import data function

In [37]:
# Put data collection code into a .py document in the src/d00_utils folder. From there 
# it can be imported into different jupyter notebooks for easy data 
def kaggle_download_raw(url, file_name):
    api.dataset_download_files(url,
                          path=external_data)
    with zipfile.ZipFile(external_data+file_name, 'r') as zipref:
        zipref.extractall(raw_data)


In [43]:
#Needs work

urls = ['mlg-ulb/creditcardfraud','dansbecker/hospital-readmissions']
file_names = ['creditcardfraud.zip','hospital-readmissions.zip']

[kaggle_download_raw(url,file_name) for url,file_name in urls, file_names]

SyntaxError: invalid syntax (<ipython-input-43-16697aeec6c2>, line 4)

#### Read in as pandas dataframes

# Data Preprocessing

# Analysis/Modeling
Do work here

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here